In [1]:
# %pip install requests

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\User\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [7]:
import requests 
false=False 
url='https://auto.ru/-/ajax/desktop/listing/'
js={"category":"cars","section":"all","has_image":false,"customs_state_group":"DOESNT_MATTER","year_to":2020,"catalog_filter":[{"mark":"BMW"}],"page":2,"geo_radius":100,"geo_id":[213]}
brheader='''
Host: auto.ru
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:96.0) Gecko/20100101 Firefox/96.0
Accept: */*
Accept-Language: ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3
Accept-Encoding: gzip, deflate, br
Referer: https://auto.ru/moskva/cars/bmw/all/?has_image=false&customs_state_group=DOESNT_MATTER&year_to=2020&page=2
x-client-app-version: 8bd42497b89
x-client-date: 1642093733401
x-csrf-token: 9acdc0d017ed1140a6b37a687a3eb6e453811ce26089d4c5
x-requested-with: XMLHttpRequest
x-page-request-id: 18045d227a4fdec2917de7935ca9602e
x-retpath-y: https://auto.ru/moskva/cars/bmw/all/?has_image=false&customs_state_group=DOESNT_MATTER&year_to=2020&page=2
x-yafp: {"a1":"+9mUFOhH1rWWmw==;0","a2":";1","a3":"RJnfx2MboJswCanp/pTWfA==;2","a4":"StYko3zFWsNlnqVqSmk8wtiCNceQDVXno+Ty8egp/wNyPQ==;3","a5":"wjYu92t/8PFxSg==;4","a6":"Vf8=;5","a7":"KvAfsVXDGAb9HQ==;6","a8":"eShrxqXZZi8JBw==;7","a9":"JW6wMh5dW7W7xg==;8","b1":"WMS1X811zcyfTA==;9","b2":"yyjQa/YhmCMLeQ==;10","b3":"naIppb8qXU+Xlw==;11","b4":"+X3co2U+4Iw=;12","b5":"ogGn4VtDnhQ22Q==;13","b6":"+z2Q4WofX4A=;14","b7":"EoTKve8VDMvjHA==;15","b8":"yUok16zaJal5sA==;16","b9":"siovMEfX8g2Gew==;17","c1":"1mBBIQ==;18","c2":"EdUNHgSksixRjnYOmi68eyuv;19","c3":"db46n2HIzPM1qzgUsfYyHffB;20","c4":"EGxT9UTLOBc=;21","c5":"idSfAP4K5Uo=;22","c6":"FJTeOQ==;23","c7":"tRonCBgZhOs=;24","c8":"M7FsDWGzADg=;25","c9":"pIwxReY43sw=;26","d1":"jPxdojxZZZeSF0iatEsue0X2DnlVMp8dIvaMllbb8LyIXVp/iZrcCU36Bg82evui8HjdaSl/;27","d2":"VDg=;28","d3":"qu1pusEc2NS/fg==;29","d4":"ykBYQZNXFv4=;30","d5":"yxfY97fzUW+BnQ==;31","d7":"RNgWvmQheUM=;32","d8":"JjI+RtXpluifFjkei3ajCpu/QyctL6CvHBo=;33","d9":"r1lNqkxAKho=;34","e1":"d3hBxv/sTectjA==;35","e2":"io2TNyuAibhRog==;36","e3":"dcX6jLdzgCIYhg==;37","e4":"zTY/oGADkWI=;38","e5":"vrzfRKzIg2ZSBA==;39","e6":"6iYhrYEdONs=;40","e7":"U+LLxN5sKjo=;41","e8":"jlUCPDEwNGA=;42","e9":"Insn4K4Iqc4=;43","f1":"r56Ndh8a2QXDkQ==;44","f2":"Qpll3AumPiI=;45","f3":"P5JubqtBinrKGQ==;46","f4":"GB3B60t7OF4=;47","f5":"Mq95O0hQKB2SGQ==;48","f6":"OK73ejIRtckzmA==;49","f7":"C7Jo/l+7Ggjvpw==;50","f8":"BH7thbLCvcJWtQ==;51","f9":"8UPV/s0K/Ak=;52","g1":"jTDuAwUJxtE=;53","g2":"n3Pb4Rhk4qQp5w==;54","g3":"sHw8QnFKhdE=;55","g4":"hanEck+CRe8=;56","g5":"VAJXAcn/KCE=;57","g6":"pMypKwIOiJhyTQ==;58","g7":"CwTU8X2eT3E=;59","g8":"4d26J5yVR4U=;60","g9":"wK7945Zt8i8=;61","h1":"9AInkX3tt0KXng==;62","h2":"l3Rdcw4wZUwafQ==;63","h3":"SR6HwaeL++QM2Q==;64","h4":"l+dC+vblTJZnEg==;65","h5":"UV8OIr2z61s=;66","h6":"X1oSmLCKI52Cfw==;67","h7":"wOjd9QM7NZNcQW/0JjWRD5T52Y3bodsWXoNp2YcH8Wyukhb1UTew/B0cvezKs9QEEap5mU2XIvCJg6uxPf9AsYXo0PUDOyKTCEEt9DM1ig+Q+eGNmqHLFlSDYtmXB6hs55JZ9UY38fwYHLzs37PcBFSqe5lGl2zwnIPlsXT/QbGT6N/1FDsnkx1BbvQiNcIPsPngjd6hxhZUg0TZiwfmbP2SU/VKN6X8Xxw=;68","h8":"/BCuzEtOychW7w==;69","h9":"FBEsYWHV4nvLaw==;70","i1":"AYXrFKDs94o=;71","i2":"Z2r3MleTRcZt9g==;72","i3":"9jhq+iOUK7PxBg==;73","i4":"9HWV3QWKr7TJlg==;74","i5":"UhPNwfrW5YtxyA==;75","z1":"X/S2w8juiivuh5tNnyXFf7ZicLUgsf6fa1fi2Mu2ToIuEE4sm+x+7345sXiWevDxY2Gp6h6fOm/8vWzuIJsEDQ==;76","z2":"89XLvCGRrsGF3dtSHbko1z7GgAO/o/OEW5WlM5ZnexjHx3hvf4rcgDMhYhLUl2gpLH82unkBpFL4IbeSMiQayw==;77","z3":"m9LfDh+/njvJ9w==;78","z6":"+NMrQoLE6YGRZuuA;79","z7":"vpnOcULUNQV8F1xf;80","z8":"Q+PA64DATB+AZg==;81","z9":"dEoDzd8viQ3P3g==;82","y1":"zkSOPfJmbbejBw==;83","y2":"GoY3Mls8TN56Ow==;84","y3":"WnalUUEBHD55VA==;85","y4":"ujyAI5y9XDRYpQ==;86","y5":"8KeCDkNpsaaNEg==;87","y6":"X/StZ62O0x6adQ==;88","y7":"BaDX/sSjTRmNPlJB;89","y8":"5xtR70OjoKUzZQ==;90","y9":"dogOajbNqQSnJg==;91","y10":"urOhyDWLxgQ20g==;92","x1":"5B3uMq7G7GmeIA==;93","x2":"8E18p8rSLsJt+A==;94","x3":"qyIHLcmgjts7Yw==;95","x4":"UqPDrKNBiZA+4A==;96","x5":"q8f9mu/g0BJ1KFS0;97","z5":"MLFA/oKKRWo=;98","z4":"poxmqqYtBdfiNOjClbQ=;99","v":"6.3.1","pgrdt":"tIPfIJoihG27AU6WOtGWU4Ft144=;100","pgrd":"5L+ttwRwCk2f75zf0ZSEywqTgH3k42gYgEvTyb/RL2pTcvJugUYEm/lGL8I60EEVEc9dfWU/bjpewbufCBOR1UgueETv/Kmglh9FrUxQFZwSqgsrZwvtV7LAp/t2eXwVf5dpqPjXBI5fretcUlR+Ah8T5itlg7QXQ7b5UuFTS5/cR43nDhNSvBc/3KMlE5082xCk2gOHclPDJOYEPKyBNznzYqM="}
content-type: application/json
Origin: https://auto.ru
Content-Length: 181
Connection: keep-alive
Cookie: autoru_sid=a%3Ag61e013872jk6kt5hq341r2qe83jehur.943df92572c6c744438057be0d0ce5f1%7C1642075015356.604800.NyNBXPK57aO3A4-Fo8IjLg.mMqAMysiCxrZlvcJeohRogrbR5jwiGpAsotk9SwB-PI; autoruuid=g61e013872jk6kt5hq341r2qe83jehur.943df92572c6c744438057be0d0ce5f1; suid=a06c2b573007272e470c6bb6f7511ff2.e5976b34552661d0ab47d79e4fc53c7e; _yasc=z7l1B/IKhjvb1ZZzk8pSZvWutbKc9J3GHbb176d99qFMpX/32Xs=; yuidlt=1; yandexuid=3644335621586616841; yandex_login=; mda2_beacon=1642093553695; crookie=ZZh3+UR44UJlHEcXIHCHEhLtuL1SWoMmxMgXvbZOjEKfcgYC+Qfo8QOA7oa+xrYCTmpAkGV1DUCRG8wwHzaMSsouZ9M=; cmtchd=MTY0MjA3NTAyNDQ0Nw==; gids=213; gradius=100; los=1; bltsr=1; _csrf_token=9acdc0d017ed1140a6b37a687a3eb6e453811ce26089d4c5; from_lifetime=1642093718731; from=direct; ys=c_chck.1432299411; X-Vertis-DC=sas; Session_id=noauth:1642093553; sso_status=sso.passport.yandex.ru:synchronized_no_beacon
Sec-Fetch-Dest: empty
Sec-Fetch-Mode: same-origin
Sec-Fetch-Site: same-origin
'''
brheader=brheader.split('\n')
brheaderdict={}
for item in brheader:
	if not(item):continue
	kv=item.split(': ')
	key,value=kv[0],kv[1]
	brheaderdict[key]=value
pass
response = requests.post(url,json=js,headers=brheaderdict)  


In [26]:
print(response.status_code)
import json
datadic=response.json()
# with open("temp.json","w") as f:
#     json.dump(datadic,f)
print(type(datadic))


200
<class 'dict'>


In [29]:
testcolumns=['bodyType', 'brand', 'car_url', 'color', 'complectation_dict',
       'description', 'engineDisplacement', 'enginePower', 'equipment_dict',
       'fuelType', 'image', 'mileage', 'modelDate', 'model_info', 'model_name',
       'name', 'numberOfDoors', 'parsing_unixtime', 'priceCurrency',
       'productionDate', 'sell_id', 'super_gen', 'vehicleConfiguration',
       'vehicleTransmission', 'vendor', 'Владельцы', 'Владение', 'ПТС',
       'Привод', 'Руль', 'Состояние', 'Таможня']

# testcolumns=['bodyType']
def deepintodict(datadic,keytofind,keyscall,rootkey="")->str:
    try:
        for key,value in datadic.items():
            if rootkey and rootkey!=key:continue #рассмотреть только один из корневых ключей
            td=type(value)
            # print('type:{0} {1}'.format(type(value),td))
            keyscall+="['{0}']".format(key)
            if td==dict:
                res=deepintodict(value,keytofind,keyscall)
                if res: return res
                continue
            if td==list:
                if len(value)>0 and type(value[0])==dict: #если список словарей, то пройти по словарям
                    count=0
                    for item in value:
                        keyscall+='[{0}]'.format(count)
                        res=deepintodict(item,keytofind,keyscall)
                        if res: return res
                        count+=1
                        continue
                else: #если список значений, то сразу проверить ключ
                    if keytofind==key:
                        return "{0}='{1}'".format(keyscall,value)
            else:
                if keytofind==key:
                    return "{0}='{1}'".format(keyscall,value)
    except Exception as ex:
        print("ошибка {0} здесь: {1}".format(ex,keyscall))
    return ""

cols=testcolumns.copy()
for col in testcolumns:
    deepkey=deepintodict(datadic,col,"datadic",rootkey='offers')
    if deepkey:
        print("{0}:  {1}".format(col,deepkey))
        cols.remove(col)
print("не найдены ключи:",cols)

color:  datadic['offers'][0][1][2][3]['availability']['category']['color_hex']['delivery_info']['description']['discount_options']['documents']['is_favorite']['old_category_id']['owner_expenses']['score']['section']['seller_type']['status']['tags']['relevance']['additional_info']['counters']['grouping_info']['hash']['id']['multiposting']['price_info']['price_history'][0]['saleId']['salon']['actual_stock']['client_id']['code']['dealer_id']['is_oficial']['logo_url']['loyalty_program']['offer_counters']['name']['phones'][0]['place']['address']['coord']['geobase_id']['region_info']['metro'][0]['rid']['name']['distance']['location']['lines'][0]['name']['color']='#ffa8af'
description:  datadic['offers'][0]['availability']['category']['color_hex']['delivery_info']['description']='Максимально бодрый M52B20 Vanos,кузов в достойном состоянии ,пороги без гнили и дыр ,есть  возрастные мелочи.Салон в комплектном состоянии ,все работает ,хлам не возили.Из того что требовало внимания ,была передняя п

In [ ]:

import requests  
response = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')  
print(response.text)

{
    "Date": "2022-01-14T11:30:00+03:00",
    "PreviousDate": "2022-01-13T11:30:00+03:00",
    "PreviousURL": "\/\/www.cbr-xml-daily.ru\/archive\/2022\/01\/13\/daily_json.js",
    "Timestamp": "2022-01-13T14:00:00+03:00",
    "Valute": {
        "AUD": {
            "ID": "R01010",
            "NumCode": "036",
            "CharCode": "AUD",
            "Nominal": 1,
            "Name": "Австралийский доллар",
            "Value": 54.5022,
            "Previous": 53.6748
        },
        "AZN": {
            "ID": "R01020A",
            "NumCode": "944",
            "CharCode": "AZN",
            "Nominal": 1,
            "Name": "Азербайджанский манат",
            "Value": 43.8897,
            "Previous": 43.8656
        },
        "GBP": {
            "ID": "R01035",
            "NumCode": "826",
            "CharCode": "GBP",
            "Nominal": 1,
            "Name": "Фунт стерлингов Соединенного королевства",
            "Value": 102.4945,
            "Previous": 101.5663
 

In [3]:
response1=requests.get('https://www.cbr-xml-daily.ru/daily.xml')
print(response1.text)

<?xml version="1.0" encoding="windows-1251"?><ValCurs Date="14.01.2022" name="Foreign Currency Market"><Valute ID="R01010"><NumCode>036</NumCode><CharCode>AUD</CharCode><Nominal>1</Nominal><Name>Àâñòðàëèéñêèé äîëëàð</Name><Value>54,5022</Value></Valute><Valute ID="R01020A"><NumCode>944</NumCode><CharCode>AZN</CharCode><Nominal>1</Nominal><Name>Àçåðáàéäæàíñêèé ìàíàò</Name><Value>43,8897</Value></Valute><Valute ID="R01035"><NumCode>826</NumCode><CharCode>GBP</CharCode><Nominal>1</Nominal><Name>Ôóíò ñòåðëèíãîâ Ñîåäèíåííîãî êîðîëåâñòâà</Name><Value>102,4945</Value></Valute><Valute ID="R01060"><NumCode>051</NumCode><CharCode>AMD</CharCode><Nominal>100</Nominal><Name>Àðìÿíñêèõ äðàìîâ</Name><Value>15,4466</Value></Valute><Valute ID="R01090B"><NumCode>933</NumCode><CharCode>BYN</CharCode><Nominal>1</Nominal><Name>Áåëîðóññêèé ðóáëü</Name><Value>29,0659</Value></Valute><Valute ID="R01100"><NumCode>975</NumCode><CharCode>BGN</CharCode><Nominal>1</Nominal><Name>Áîëãàðñêèé ëåâ</Name><Value>43,7429<

In [4]:
currencies = response.json()  
print(currencies)  

{'Date': '2022-01-14T11:30:00+03:00', 'PreviousDate': '2022-01-13T11:30:00+03:00', 'PreviousURL': '//www.cbr-xml-daily.ru/archive/2022/01/13/daily_json.js', 'Timestamp': '2022-01-13T14:00:00+03:00', 'Valute': {'AUD': {'ID': 'R01010', 'NumCode': '036', 'CharCode': 'AUD', 'Nominal': 1, 'Name': 'Австралийский доллар', 'Value': 54.5022, 'Previous': 53.6748}, 'AZN': {'ID': 'R01020A', 'NumCode': '944', 'CharCode': 'AZN', 'Nominal': 1, 'Name': 'Азербайджанский манат', 'Value': 43.8897, 'Previous': 43.8656}, 'GBP': {'ID': 'R01035', 'NumCode': '826', 'CharCode': 'GBP', 'Nominal': 1, 'Name': 'Фунт стерлингов Соединенного королевства', 'Value': 102.4945, 'Previous': 101.5663}, 'AMD': {'ID': 'R01060', 'NumCode': '051', 'CharCode': 'AMD', 'Nominal': 100, 'Name': 'Армянских драмов', 'Value': 15.4466, 'Previous': 15.4943}, 'BYN': {'ID': 'R01090B', 'NumCode': '933', 'CharCode': 'BYN', 'Nominal': 1, 'Name': 'Белорусский рубль', 'Value': 29.0659, 'Previous': 29.0353}, 'BGN': {'ID': 'R01100', 'NumCode': 

In [5]:
currencies['Valute']['UAH']  
print(currencies['Valute']['CZK']['Name'])

Чешских крон


In [6]:
def exchange_rates(currency, format='full'):    
    url = 'https://www.cbr-xml-daily.ru/daily_json.js'  
    response = requests.get(url).json()['Valute']    
    data = response[currency]     
    if format == 'full':    
        return data      
    elif format == 'value':    
        return data['Value']    

def currency_name(ID):
    url = 'https://www.cbr-xml-daily.ru/daily_json.js'  
    response = requests.get(url).json()['Valute']    
    for key,value in response.items():
        curdata=value
        if curdata['ID']==ID:
            return curdata['Name']
    return None

print(currency_name('R01700J'))

Турецких лир


In [7]:
import requests  
  
url = 'https://nplus1.ru/news/2019/06/04/slothbot'   
  
response = requests.get(url)  
  
# Убедимся, что мы успешно получили ответ     
print(response.status_code)    
# => 200   
  
# Выведем полученные данные    
print(response.text)   

200
<!doctype html>
<html class="no-js bg-fixed _no-bg" style="background-image:url(https://nplus1.ru/images/2019/06/04/b32b62189fb87cce895e229e1d6d27b4.jpeg)" lang="">
<head>
    
    <meta charset="utf-8">
    <meta http-equiv="x-ua-compatible" content="ie=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1">
        <meta name="apple-mobile-web-app-capable" content="yes">
    <meta name="apple-mobile-web-app-status-bar-style" content="black">
                <meta name='wmail-verification' content='7991d7eb02d759f05b9050e111a7e3eb' />

    <link rel="canonical" href="https://nplus1.ru/news/2019/06/04/slothbot" />

        <title>Робота-ленивца научили лазать по паутине из тросов</title>

    	    <meta itemprop="datePublished" content="2019-06-04"/>
	
	    <meta name="mediator_author" content="Григорий Копиев"/> 
	
        <!-- amp page -->
    <link rel="amphtml" href="https://nplus1.ru/news/2019/06/04/slothbot/amp">
    

        <!-- for Google -->
     

In [8]:
# %pip install beautifulsoup4  

Note: you may need to restart the kernel to use updated packages.


In [9]:
# Импортируем наши библиотеки    
from bs4 import BeautifulSoup    
import requests    
    
# Получаем данные, как и ранее    
url = 'https://nplus1.ru/news/2019/06/04/slothbot'   
response = requests.get(url)    
    
# Теперь создадим объект BeautifulSoup, указывая html парсер    
page = BeautifulSoup(response.text, 'html.parser')    
    
# Всё готово, чтобы получать данные из страницы    
# Для начала получим title, отображающийся на закладках браузера    
print(page.title)  
# => <title>Робота-ленивца научили лазать по паутине из тросов</title>  
    
# Мы получили тэг. Чтобы достать из него текст, вызовем атрибут text    
page.title.text    
# => 'Робота-ленивца научили лазать по паутине из тросов'  

<title>Робота-ленивца научили лазать по паутине из тросов</title>


'Робота-ленивца научили лазать по паутине из тросов'

In [10]:
def wiki_header(url):
    response = requests.get(url)    
    page = BeautifulSoup(response.text, 'html.parser')  
    qq=page.find('h1')
    return qq.text
    
wiki_header('https://en.wikipedia.org/wiki/Operating_system')

'Operating system'

In [11]:

url = 'https://en.wikipedia.org/wiki/List_of_programming_languages'  
    
response = requests.get(url)  
page = BeautifulSoup(response.text, 'html.parser')  
page.find('a')  
# => <a id="top"></a>  


links = page.find_all('a')  
# Посмотрим, сколько всего мы получили  
print(len(links))
# => 935
    
# Посмотрим на некоторые из ссылок  
print([link.text for link in links[500:510]])  
# => ['Oxygene', 'Oz', 'edit', 'P', 'P4', 'P′′', 'ParaSail (programming language)', 'PARI/GP', 'Pascal', 'Pascal Script']


930
['OptimJ', 'Orc', 'ORCA/Modula-2', 'Oriel', 'Orwell', 'Oxygene', 'Oz', 'edit', 'P', 'P4']


In [12]:

# Получаем все элементы с тегом 'div' и классом 'div-col'  
all_blocks = page.find_all('div', class_='div-col')  
    
# Выбираем первый по счету блок  
first_block = all_blocks[0]  
# Берём оттуда ссылки (ограничимся первыми десятью)
links = first_block.find_all('a')
print([link.text for link in links[:10]])
# => ['1C:Enterprise programming language']


['A.NET (A#/A sharp)', 'A-0 System', 'A+ (A plus)', 'ABAP', 'ABC', 'ABC ALGOL', 'ACC', 'Accent (Rational Synergy)', 'Ace DASL (Distributed Application Specification Language)', 'Action!']


In [13]:
def get_actors(url):
    response = requests.get(url)  
    page = BeautifulSoup(response.text, 'html.parser')  
    act = page.find('div', class_='styles_actors__2zt1j')  
    acts=act.find_all('a',class_='styles_link__1dkjp')
    qq=[]
    for act in acts:
        qq.append(act.text)
    return qq

# get_actors('https://www.kinopoisk.ru/film/42326/')
get_actors('https://www.kinopoisk.ru/film/326/')

['Тим Роббинс',
 'Морган Фриман',
 'Боб Гантон',
 'Уильям Сэдлер',
 'Клэнси Браун',
 'Гил Беллоуз',
 'Марк Ролстон',
 'Джеймс Уитмор',
 'Джеффри ДеМанн',
 'Ларри Бранденбург']

In [17]:
# %pip install lxml
# %pip install pandas
# %pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
url = 'https://www.cbr.ru/key-indicators/'  
# Таблица с драгметаллами оказалась третьей по счёту  
qq=pd.read_html(url)
print(qq[1])


                 0           1           2
0           валюта  13.01.2022  14.01.2022
1  Доллар США  USD      745277      745686
2        Евро  EUR      846709      854556


In [2]:
from bs4 import BeautifulSoup  
import pandas as pd  
import requests  
    
url = 'https://www.banki.ru/banks/ratings/'
soup = BeautifulSoup(requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}).text, 'html.parser')  

all_blocks = soup.find_all('table', class_='standard-table standard-table--row-highlight margin-bottom-small margin-top-x-small')
data = all_blocks[0]
df = pd.read_html(str(data))[0]  


In [3]:
qq='bfc62b39bfc62b39bfc62b3964bfbceed9bbfc6bfc62b39de1caa2ac943f4855e92bd90'



In [4]:
import requests  
  
url = 'https://api.vk.com/method/users.get'   
params = {'user_id': 1, 'v': 5.95, 'fields': 'sex,bdate', 'access_token': qq, 'lang': 'ru'}  
  
# Мы можем выставить параметры запроса через аргумент params  
response = requests.get(url, params=params)  
response.text  

'{"response":[{"id":1,"first_name":"Павел","last_name":"Дуров","can_access_closed":true,"is_closed":false,"sex":2,"bdate":"10.10.1984"}]}'

In [5]:

from pprint import pprint  
    
pprint(response.json())  

    
user = response.json()['response'][0]  
    
# Выведем дату рождения  
print(user['bdate'])  
# => 10.10.1984  
    
# Выведем имя  
print(user['first_name'])  
# => Павел  


{'response': [{'bdate': '10.10.1984',
               'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров',
               'sex': 2}]}
10.10.1984
Павел


In [6]:
ids = ",".join(map(str, range(1, 4)))  
print(ids)  
# => 1,2,3  
  
params = {'user_ids': ids, 'v': 5.95, 'fields': 'bday', 'access_token': qq, 'lang': 'ru'}  
  
pprint(requests.get(url, params=params).json()) 

1,2,3
{'response': [{'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров'},
              {'can_access_closed': False,
               'first_name': 'Александра',
               'id': 2,
               'is_closed': True,
               'last_name': 'Владимирова'},
              {'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 3,
               'last_name': ''}]}


In [7]:
ids = ",".join(map(str, range(1, 501)))  
params = {'user_ids': ids, 'v': 5.95, 'fields': 'sex', 'access_token': qq}  
userdics=requests.get(url, params=params).json()['response'] 
print(userdics[0])

{'id': 1, 'first_name': 'Pavel', 'last_name': 'Durov', 'can_access_closed': True, 'is_closed': False, 'sex': 2}


In [8]:
df=pd.DataFrame.from_dict(userdics)
df.head(3)

,id,first_name,last_name,can_access_closed,is_closed,sex,deactivated
0,1,Pavel,Durov,True,False,2,NaN
1,2,Alexandra,Vladimirova,False,True,1,NaN
2,3,DELETED,,NaN,NaN,0,deleted


In [9]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 500 non-null    int64 
 1   first_name         500 non-null    object
 2   last_name          500 non-null    object
 3   can_access_closed  364 non-null    object
 4   is_closed          364 non-null    object
 5   sex                500 non-null    int64 
 6   deactivated        136 non-null    object
dtypes: int64(2), object(5)
memory usage: 27.5+ KB


In [10]:
len(df.index)
df[df.sex==1].sample(5)

,id,first_name,last_name,can_access_closed,is_closed,sex,deactivated
259,260,Anastasia,Grafova,True,False,1,NaN
202,203,Zoya,Barabanova,True,False,1,NaN
145,146,Alexandra,Muromtseva,True,False,1,NaN
354,355,Elena,Batsmanova,True,False,1,NaN
417,418,Yulia,Ernie,True,False,1,NaN


In [11]:
print(len(df[df.sex==1].index)/len(df[df.sex!=0].index))

0.4821852731591449


In [12]:

import requests  
url = 'https://api.vk.com/method/groups.getMembers'  
params = {  
    'group_id': 'vk',  
    'v': 5.95,  
    'access_token': qq  
}  
response = requests.get(url, params = params)  
data = response.json()  
print(data)  
# => {'response': {'count': 11240724, 'items': [5, 6, 10, 19, 34, 47, 54, 79, 177, 193, 205, 219... 
    
len(data['response']['items'])  
# => 1000


{'response': {'count': 12241022, 'items': [5, 6, 19, 34, 47, 54, 79, 177, 193, 198, 212, 219, 239, 243, 254, 345, 404, 406, 407, 467, 485, 510, 550, 619, 628, 640, 643, 690, 702, 720, 721, 724, 744, 804, 809, 831, 832, 834, 847, 900, 905, 907, 914, 943, 952, 958, 966, 976, 979, 1000, 1018, 1023, 1032, 1033, 1038, 1039, 1059, 1097, 1131, 1139, 1140, 1159, 1174, 1185, 1188, 1290, 1301, 1333, 1334, 1336, 1351, 1381, 1386, 1388, 1406, 1411, 1418, 1432, 1494, 1531, 1543, 1568, 1586, 1590, 1593, 1598, 1610, 1615, 1632, 1634, 1650, 1679, 1690, 1697, 1698, 1699, 1700, 1721, 1740, 1754, 1796, 1814, 1820, 1829, 1834, 1839, 1840, 1843, 1858, 1863, 1868, 1869, 1887, 1889, 1917, 1943, 1947, 1955, 1969, 2019, 2028, 2050, 2051, 2052, 2059, 2077, 2103, 2145, 2150, 2195, 2201, 2202, 2230, 2236, 2273, 2281, 2294, 2296, 2298, 2376, 2389, 2395, 2403, 2412, 2436, 2456, 2466, 2470, 2484, 2515, 2527, 2539, 2571, 2576, 2592, 2601, 2622, 2644, 2654, 2692, 2706, 2745, 2755, 2767, 2787, 2797, 2827, 2858, 2896, 2

1000

In [13]:
users_for_checking = data['response']['items'][:20]  
print(users_for_checking)  

[5, 6, 19, 34, 47, 54, 79, 177, 193, 198, 212, 219, 239, 243, 254, 345, 404, 406, 407, 467]


In [14]:
import time
cycle=0
count = 1000  
offset = 0  
user_ids = []
max_count = 100000  
while offset < max_count:  
    cycle+=1
    # print('Выгружаю {} пользователей с offset = {}'.format(count, offset))     
    print(cycle,sep=' ',end='')     
    params = {  
        'group_id': 'vk',  
        'v': 5.131,  
        'count': count,  
        'offset': offset,  
        'access_token': qq  
    }     
    # такой же запрос как в прошлый раз  
    r = requests.get(url, params = params)  
    data = r.json()     
    user_ids += data['response']['items']   
    
    # увеличиваем смещение на количество строк выгрузки  
    offset += count  
    time.sleep(0.5)
        
print('\ndone ',user_ids[99999])  
    
# user_ids == users_for_checking  


123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100
done  6052530


In [15]:
import requests  
from pprint import pprint  
  
url = 'https://api.vk.com/method/wall.get'  
params = {  
    'domain': 'vk',  
    'filter': 'owner',  
    'count': 10,  
    'offset': 0,  
    'access_token': qq,  
    'v': 5.95  
}  
response = requests.get(url, params = params)  
response.json()


{'response': {'count': 464,
  'items': [{'id': 1268470,
    'from_id': -22822305,
    'owner_id': -22822305,
    'date': 1640980531,
    'marked_as_ads': 0,
    'post_type': 'post',
    'text': '',
    'copy_history': [{'id': 393,
      'owner_id': 92933,
      'from_id': 92933,
      'date': 1640980164,
      'post_type': 'post',
      'text': 'Команда ВКонтакте — люди, которые искренне восхищают. В этом году мы сделали рекордное число технологических прорывов и продуктовых обновлений за всю историю социальной сети. \n\nВ уходящем 2021-м мы не только отметили 15-летие ВКонтакте и громко обновились на iPad, но и представили больше 230 крупных нововведений — в том числе неограниченные по числу участников VK Звонки, масштабную библиотеку контента и новые технологии платформы VK Видео, новые рекомендации Клипов, VK Музыки, представили новые алгоритмы Ленты Новостей и Рекомендаций, редизайн вкладки «Сервисы» (VK SuperApp), огромное число обновлений в мессенджере, одними из первых в мире те

In [16]:
response.json()['response']['items']

[{'id': 1268470,
  'from_id': -22822305,
  'owner_id': -22822305,
  'date': 1640980531,
  'marked_as_ads': 0,
  'post_type': 'post',
  'text': '',
  'copy_history': [{'id': 393,
    'owner_id': 92933,
    'from_id': 92933,
    'date': 1640980164,
    'post_type': 'post',
    'text': 'Команда ВКонтакте — люди, которые искренне восхищают. В этом году мы сделали рекордное число технологических прорывов и продуктовых обновлений за всю историю социальной сети. \n\nВ уходящем 2021-м мы не только отметили 15-летие ВКонтакте и громко обновились на iPad, но и представили больше 230 крупных нововведений — в том числе неограниченные по числу участников VK Звонки, масштабную библиотеку контента и новые технологии платформы VK Видео, новые рекомендации Клипов, VK Музыки, представили новые алгоритмы Ленты Новостей и Рекомендаций, редизайн вкладки «Сервисы» (VK SuperApp), огромное число обновлений в мессенджере, одними из первых в мире технологию ускорение работы ВКонтакте по протоколу QUIC, а также 

In [17]:
response.json()['response']['items'][0]

{'id': 1268470,
 'from_id': -22822305,
 'owner_id': -22822305,
 'date': 1640980531,
 'marked_as_ads': 0,
 'post_type': 'post',
 'text': '',
 'copy_history': [{'id': 393,
   'owner_id': 92933,
   'from_id': 92933,
   'date': 1640980164,
   'post_type': 'post',
   'text': 'Команда ВКонтакте — люди, которые искренне восхищают. В этом году мы сделали рекордное число технологических прорывов и продуктовых обновлений за всю историю социальной сети. \n\nВ уходящем 2021-м мы не только отметили 15-летие ВКонтакте и громко обновились на iPad, но и представили больше 230 крупных нововведений — в том числе неограниченные по числу участников VK Звонки, масштабную библиотеку контента и новые технологии платформы VK Видео, новые рекомендации Клипов, VK Музыки, представили новые алгоритмы Ленты Новостей и Рекомендаций, редизайн вкладки «Сервисы» (VK SuperApp), огромное число обновлений в мессенджере, одними из первых в мире технологию ускорение работы ВКонтакте по протоколу QUIC, а также автоматически

In [18]:
stats = {}  
              
for record in response.json()['response']['items'][:]:  
    title = record['text'][:30]  
    if title:  
        stats[title] = [record['comments']['count'], record['likes']['count'], record['reposts']['count'], record['date'] ]  
pprint(stats)  

{'А если взглянуть в целом, то к': [272, 849, 255, 1640094467],
 'ВКонтакте всегда есть что посм': [196, 873, 87, 1640017040],
 'Запускаем собственное шоу в пе': [71, 1277, 452, 1640940328],
 'Каким был ваш 2021-й ВКонтакте': [591, 2103, 9749, 1640174402],
 'Мы приготовили для вас целую г': [168, 3903, 390, 1640712137],
 'Объявляем итоги первого розыгр': [591, 1289, 327, 1640871560],
 'Одни называют всё, что происхо': [126, 1072, 284, 1639491162],
 'Приближаем праздники вместе — ': [7041, 1160, 676, 1639590427],
 'Эти каникулы будут особенными ': [128, 1150, 153, 1640171709]}


In [19]:
import requests  
from pprint import pprint  

def get_items(gr='vk', token=''):
  
    url = 'https://api.vk.com/method/wall.get'  
    params = {  
    'domain': 'vk',  
    'filter': 'owner',  
    'count': 10,  
    'offset': 0,  
    'access_token': token,  
    'v': 5.95  
    }  
    response = requests.get(url, params = params)  
    dicts=response.json()['response']
    return dicts

ds=get_items(gr='vk', token=qq)
print(ds['count'])
print(ds['items'][0])
def get_smm_index(gr='vk', token=''):
    sum=0              
    for record in ds['items'][:10]:  
        sum+= record['comments']['count']+ record['likes']['count']+ record['reposts']['count'] 
    return sum/ds['count']


464
{'id': 1268470, 'from_id': -22822305, 'owner_id': -22822305, 'date': 1640980531, 'marked_as_ads': 0, 'post_type': 'post', 'text': '', 'copy_history': [{'id': 393, 'owner_id': 92933, 'from_id': 92933, 'date': 1640980164, 'post_type': 'post', 'text': 'Команда ВКонтакте — люди, которые искренне восхищают. В этом году мы сделали рекордное число технологических прорывов и продуктовых обновлений за всю историю социальной сети. \n\nВ уходящем 2021-м мы не только отметили 15-летие ВКонтакте и громко обновились на iPad, но и представили больше 230 крупных нововведений — в том числе неограниченные по числу участников VK Звонки, масштабную библиотеку контента и новые технологии платформы VK Видео, новые рекомендации Клипов, VK Музыки, представили новые алгоритмы Ленты Новостей и Рекомендаций, редизайн вкладки «Сервисы» (VK SuperApp), огромное число обновлений в мессенджере, одними из первых в мире технологию ускорение работы ВКонтакте по протоколу QUIC, а также автоматические переводы и автос

In [20]:
import requests  

def get_smm_index(gr,token):
    url = 'https://api.vk.com/method/groups.getMembers'  
    params = {  
        'group_id': gr,  
        'v': 5.95,  
        'access_token': token  
    }  
    response = requests.get(url, params = params)  
    membercount = response.json()['response']['count']  

    url = 'https://api.vk.com/method/wall.get'  
    params = {  
    'domain': gr,  
    'filter': 'owner',  
    'count': 10,  
    'offset': 0,  
    'access_token': token,  
    'v': 5.95  
    }  
    response = requests.get(url, params = params)  
    ds=response.json()['response']
    sum=0              
    for record in ds['items'][:10]:  
        sum+= record['comments']['count']+ record['likes']['count']+ record['reposts']['count'] 
    return sum/membercount

token = qq
get_smm_index('vk', token)

0.003020416259157319

In [21]:
sum=0     
count=0         
for record in ds['items'][:10]:
    sum+= record['comments']['count']+ record['likes']['count']+ record['reposts']['count'] 
    count+=1
    print(count,sum)  
pprint(sum/464)  

1 1740
2 3540
3 5747
4 10208
5 22651
6 24082
7 25458
8 26614
9 35491
10 36973
79.68318965517241


In [22]:
requests.get("http://www.google.com").status_code 

200